In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

houses=pd.read_csv("C:\\Users\\augusto\\.vscode\\Data Sci and Ml\\Linear_Regression\\datasets\\housing\\housing.csv",encoding="utf-8")
houses

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [7]:
#CONFIGURATIONS



from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.base import TransformerMixin,BaseEstimator

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedShuffleSplit


imput1=SimpleImputer(strategy="median")
class1=OrdinalEncoder()
class2=OneHotEncoder() # -> use for binary



#custom transform [density]
class CobbineATB(BaseEstimator,TransformerMixin):
    def __init__(self,config1=True):
        self.config1=config1

    def fit(self,X,y=None):
        return self
    
    def transform(self,X): #we will use nums as labels
        t_rooms,pop,med_house=3,5,7

        rooms_per_pop=X[:,pop] / X[:,t_rooms]
        median_pop_price=X[:,med_house] / X[:,pop]
        
        if self.config1:
            return np.c_[X,median_pop_price,rooms_per_pop]#-> integrated atributes
        else:
            return np.c_[X,median_pop_price,rooms_per_pop]
    
print(houses[['median_income','housing_median_age','households']].corr())


                    median_income  housing_median_age  households
median_income            1.000000           -0.119034    0.013033
housing_median_age      -0.119034            1.000000   -0.302916
households               0.013033           -0.302916    1.000000


In [3]:
#Train Sets


houses["income_ranges"]=pd.cut(houses["median_income"],
                               bins=[0.,1.5,3.0,4.5,6.,np.inf],
                               labels=[1,2,3,4,5])


splits=StratifiedShuffleSplit(n_splits=1,test_size=0.25,random_state=22)
for x_train, x_test in splits.split(houses, houses["income_ranges"]):
    strat_x_train=houses.loc[x_train]
    strat_test=houses.loc[x_test]

for set_ in (strat_x_train, strat_test):
    set_.drop("income_ranges", axis=1, inplace=True)

#Representative Data
housees=strat_x_train.copy()
houses_train=strat_x_train.drop("median_house_value",axis=1)
houses_labels=strat_x_train["median_house_value"].copy()
#print(housees)

housing_atributes_nums=houses_train.drop("ocean_proximity",axis=1) #Using on pipelines
housing_classes=houses_train[["ocean_proximity"]] #using on pipeines


nums_columns=list(housing_atributes_nums)
text_columns=["ocean_proximity"]





In [4]:
#Pipelines and Data Transform

nums_pip=Pipeline([
    ("imputs",SimpleImputer(strategy="median")),
    ("combine",CobbineATB()),
    ("std",StandardScaler()),
])

full_transform=ColumnTransformer([
    ("pip num",nums_pip,nums_columns), #OCEAN : 12
    ("classl",OneHotEncoder(),text_columns), #onehot because the data will be compared and don't nned to be random nums
])

#print(houses_train)
data_clean=full_transform.fit_transform(houses_train) # first fit,so it fit the pipeline model
dasta=full_transform.transform(houses_train.iloc[:5]) # second based on first fit
hj=pd.DataFrame(data_clean)
print(pd.DataFrame(data_clean))
print(pd.DataFrame(dasta))


             0         1         2         3         4         5         6   \
0     -1.372412  1.079423  1.462451 -0.332738 -0.605962 -0.632769 -0.587437   
1      0.716810 -0.884800  0.827746 -0.294906  0.223683 -0.486897  0.166427   
2     -0.592702  0.845029 -0.838355 -0.291303 -0.334141 -0.087035 -0.236322   
3     -1.832241  1.435702  0.748408 -0.535861 -0.539779 -0.742602 -0.843027   
4      0.741801 -0.800418  0.589732 -0.479563 -0.511416 -0.196868 -0.411880   
...         ...       ...       ...       ...       ...       ...       ...   
15475 -1.432390  1.004417  1.224437 -0.497578 -0.485415 -0.749467 -0.455769   
15476  0.626843 -0.687909  1.462451 -0.027378 -0.175776 -0.129081 -0.045275   
15477 -1.422393  1.004417 -0.917693 -0.627289 -0.473597 -0.292972 -0.306029   
15478 -0.882594  1.313817 -1.155708  0.250059 -0.175776  0.003921 -0.120145   
15479 -1.567339  1.309129  0.748408 -0.432723 -0.031592 -0.311850  0.068321   

             7         8         9    10   11   12 

In [ ]:
#MODEL TRAINING

from sklearn.linear_model import LinearRegression
import joblib


model1=LinearRegression()
model1.fit(data_clean,houses_labels)

data_test=houses_train.iloc[:9]
print(data_test)
data_predict=houses_labels.iloc[:9]
data_test_clean=full_transform.transform(data_test)
print(pd.DataFrame(data_test_clean))

x1=model1.predict(data_test_clean)

print(f"predictions: {model1.predict(data_test_clean)} \n")
print(f"labels(compare): {data_predict} \n")



       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
1819     -122.32     37.94                47.0       1911.0           283.0   
8295     -118.14     33.75                39.0       1995.0           634.0   
9534     -120.76     37.44                18.0       2003.0           398.0   
19363    -123.24     38.70                38.0       1460.0           311.0   
7041     -118.09     33.93                36.0       1585.0           323.0   
14951    -116.96     32.71                18.0       2413.0           533.0   
6003     -117.77     34.08                27.0       5929.0           932.0   
9320     -122.53     37.97                52.0       1560.0           451.0   
10567    -117.74     33.73                18.0        328.0            68.0   

       population  households  median_income ocean_proximity  
1819        697.0       275.0         6.2712        NEAR BAY  
8295        867.0       567.0         4.0795      NEAR OCEAN  
9534       1333.0   

In [8]:
#Accuracy simple test


from sklearn.metrics import root_mean_squared_error,mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score
print(houses_labels.items)

def predicts_accuracy(waite,predict,method="vvv",data=data_test_clean):
    cross_val=cross_val_score(model1,data,predict,scoring="r2",cv=4)
    rmse=root_mean_squared_error(waite,predict)
    mae=mean_absolute_error(waite,predict)
    r2=r2_score(waite,predict)

    stats={"mean":cross_val.mean(),"deviation":cross_val.std()}
    
    return rmse,cross_val,mae,r2,stats


x1=np.array(x1)
data_predict=np.array(data_predict) #convert to test in accuracy
predicts_accuracy(data_predict,x1) #labels,model-prevision

NameError: name 'houses_labels' is not defined

In [7]:
#Hyperparameters with grid search


from sklearn.model_selection import GridSearchCV
mode=LinearRegression

parameters_config1=[{'fit_intercept':[False],'copy_X':[False],'n_jobs':[1,100,-1],'positive':[True]},
                    {'fit_intercept':[False],'copy_X':[True],'n_jobs':[1,100,-1],'positive':[True]},
                    {'fit_intercept':[False],'copy_X':[False],'n_jobs':[1,100,-1],'positive':[False]},
                    {'fit_intercept':[True],'copy_X':[False],'n_jobs':[1,100,-1],'positive':[True]},
                    {'fit_intercept':[True],'copy_X':[True],'n_jobs':[1,100,-1],'positive':[True]},
                    {'fit_intercept':[False],'copy_X':[True],'n_jobs':[1,100,-1],'positive':[False]},
                    {'fit_intercept':[True],'copy_X':[False],'n_jobs':[1,100,-1],'positive':[False]}]

grid1=GridSearchCV(model1,parameters_config1,scoring="r2",return_train_score=True)

grid1.fit(data_clean,houses_labels)

#Avaliables:
#best config:
print({"best parans":grid1.best_params_,
       "best estimators": grid1.best_estimator_})

results=grid1.cv_results_
for score,params in zip(results["mean_test_score"],results["params"]): #acess results dictionary
    print(-score,params) #because there's not positive version

#just for random forest
#feature_importance=grid1.best_estimator_.feature_importances_

{'best parans': {'copy_X': False, 'fit_intercept': False, 'n_jobs': 1, 'positive': False}, 'best estimators': LinearRegression(copy_X=False, fit_intercept=False, n_jobs=1)}
-0.6090882188867921 {'copy_X': False, 'fit_intercept': False, 'n_jobs': 1, 'positive': True}
-0.6090882188867921 {'copy_X': False, 'fit_intercept': False, 'n_jobs': 100, 'positive': True}
-0.6090882188867921 {'copy_X': False, 'fit_intercept': False, 'n_jobs': -1, 'positive': True}
-0.6090882188867921 {'copy_X': True, 'fit_intercept': False, 'n_jobs': 1, 'positive': True}
-0.6090882188867921 {'copy_X': True, 'fit_intercept': False, 'n_jobs': 100, 'positive': True}
-0.6090882188867921 {'copy_X': True, 'fit_intercept': False, 'n_jobs': -1, 'positive': True}
-0.6458537349020752 {'copy_X': False, 'fit_intercept': False, 'n_jobs': 1, 'positive': False}
-0.6458537349020752 {'copy_X': False, 'fit_intercept': False, 'n_jobs': 100, 'positive': False}
-0.6458537349020752 {'copy_X': False, 'fit_intercept': False, 'n_jobs': -1, 

In [ ]:
#Using better configs:
from scipy import stats

main_model=grid1.best_estimator_
main_data=full_transform.transform(houses_train)
main_labels=houses_labels
results=main_model.predict(main_data) #because already full with data and features
mse=mean_squared_error(main_labels,results)
r2=r2_score(main_labels,results)
print(f"r2: {r2},mse: {mse}")



r2: 0.6495277843906254,mse: 4672513161.427211


In [9]:
#generalization
confidence=0.9

def cof(confidence,results,main_labels):
    squared_errors=(results-main_labels)**2
    st=np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,
            loc=squared_errors.mean(),
                scale=stats.sem(squared_errors)))
    
    return st
   

print(cof(confidence,results,main_labels))

[67227.24014114 69465.99532045]


In [10]:
#Save model

model_save1=joblib.dump(main_model,"Linear_house.joblib") #can be the grid object with all options
model_use1=joblib.load("Linear_house.joblib")
#load for use an file model